# oak-d × spectacularAI × nerfstudio Google Colab ワークフロー

このノートブックは、oak-dで撮影したデータ（spectacularAI形式）をGoogle Colabにアップロードし、nerfstudioで学習・可視化・ダウンロードするまでの一連の手順をまとめたものです。

---

## 概要
1. Colab環境セットアップ
2. データアップロード
3. nerfstudioインストール
4. spectacularAIデータをnerfstudio用に変換（必要なら）
5. nerfstudioで学習
6. 結果ダウンロード


In [ ]:
# 1. Colab環境セットアップ
!python --version
!nvidia-smi  # GPU確認
!pip install --upgrade pip

# GPU対応PyTorchのインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install nerfstudio

# GPU利用可能性をチェック
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

In [ ]:
# 2. データをGoogle Colabにアップロード
# processed/ フォルダをzipファイルにしてアップロードしてください
# ローカルで: processed フォルダを右クリック → 圧縮 → processed.zip

from google.colab import files
import zipfile
import os

# zipファイルをアップロード
print("processed.zip ファイルをアップロードしてください（NerfStudio形式のデータ）")
uploaded = files.upload()

# zipファイルを展開
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"{filename} を展開しました")
        os.remove(filename)  # zipファイルを削除

# アップロードされたディレクトリ構造を確認
!ls -la

In [ ]:
# nerfstudioのインストール確認（セル1で既にインストール済み）
import nerfstudio
print('nerfstudio version:', nerfstudio.__version__)

# nerfstudioのコマンドが使用可能か確認
!ns-train --help | head -5

In [ ]:
# 3. アップロードしたデータの確認
# processed.zipを展開済みなので、NerfStudio形式データを確認
print("アップロードされたデータ構造:")
!find processed -type f | head -20

# transforms.jsonの存在確認
import os
if os.path.exists('processed/transforms.json'):
    print("✓ transforms.json が見つかりました")
else:
    print("✗ transforms.json が見つかりません")

# 画像フォルダの確認
if os.path.exists('processed/images'):
    image_count = len([f for f in os.listdir('processed/images') if f.endswith('.jpg')])
    print(f"✓ 画像フォルダに {image_count} 枚の画像があります")
else:
    print("✗ images フォルダが見つかりません")

In [ ]:
# 4. nerfstudioで学習
# processed/ フォルダのデータを使って学習を実行
!ns-train nerfacto --data processed/ --output-dir outputs/

# 学習が完了すると outputs/ フォルダに結果が保存されます
print("学習完了後、outputs/ フォルダを確認してください")

In [ ]:
# 5. 学習結果のダウンロード
from google.colab import files
import glob
import os

# outputs/ ディレクトリ内のファイル構造を確認
print("学習結果:")
!find outputs -name "*.ckpt" -o -name "*.json" -o -name "config.yml" | head -10

# 学習済みモデル（チェックポイント）をダウンロード
for file in glob.glob('outputs/**/nerfstudio_models/**/*.ckpt', recursive=True):
    if os.path.isfile(file):
        print(f"ダウンロード中: {file}")
        files.download(file)

# 設定ファイルもダウンロード
for file in glob.glob('outputs/**/config.yml', recursive=True):
    if os.path.isfile(file):
        print(f"ダウンロード中: {file}")
        files.download(file)